https://www.pinecone.io/learn/series/faiss/faiss-tutorial/ (ref)

In [2]:
import requests
from io import StringIO
import pandas as pd

In [3]:
url = 'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt'
res = requests.get(url)
data_df = pd.read_csv(StringIO(res.text), sep='\t')
data_df.head(2)

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
